# Python - Generic Types

---

Python offers 4 complementary typing approaches that work together. 

In [1]:
from typing import Any, Generic, List, TypeVar  # noqa: F401

## Duck Typing

Duck Typing is dynamic runtime polymorphism: Python trusts that objects provide needed methods. No explicit type declarations. Behavior-driven.

In *runtime polymorphism*, different object types can be used interchangeably as long as they provide the required methods or behaviors. 

This is exactly what duck typing is: if an object "quacks" (has the methods you want), it can be treated like a duck (used as that type), no matter the actual class.

In [2]:
class Duck:
    def quack(self):
        return "Quack!"


class Person:
    def quack(self):
        return "I'm quacking like a duck!"


def join_quacks(quackers: List[Any]):
    return " ".join([q.quack() for q in quackers])


duck = Duck()
person = Person()

sounds = join_quacks([duck, person])
sounds

"Quack! I'm quacking like a duck!"

## Generics

* Generics are static typed *parameterization*, allowing classes/functions to be defined over arbitrary types with type variables, improving type safety and reusability.
* Provide static type checkers like `mypy` with more precise information for verifying code correctness before runtime.
* Document your code's intent more clearly by specifying which types can be used with your generic components.

In [3]:
%load_ext nb_mypy
%nb_mypy On

Version 1.0.6


In [4]:
T = TypeVar("T")


class Stack(Generic[T]):
    def __init__(self):
        self._items = []

    def push(self, item: T):
        self._items.append(item)

    def pop(self) -> T:
        return self._items.pop()


int_stack = Stack[int]()
int_stack.push(42)

# Now try pushing a string
int_stack.push("not an int")  # mypy will flag this line

<cell>1: error: Name "TypeVar" is not defined  [name-defined]


<cell>1: note: Did you forget to import it from "typing"? (Suggestion: "from typing import TypeVar")


<cell>4: error: Name "Generic" is not defined  [name-defined]


<cell>5: error: Function is missing a return type annotation  [no-untyped-def]


<cell>5: note: Use "-> None" if function does not return a value


...

<cell>8: note: See https://mypy.readthedocs.io/en/stable/common_issues.html#variables-vs-type-aliases


<cell>11: error: Variable "__main__.T" is not valid as a type  [valid-type]


<cell>11: note: See https://mypy.readthedocs.io/en/stable/common_issues.html#variables-vs-type-aliases


<cell>12: error: Returning Any from function declared to return T?  [no-any-return]


<cell>15: error: The type "type[Stack]" is not generic and not indexable  [misc]


In [5]:
%nb_mypy Off

## Protocols

Protocols formalize duck typing for static type *checking*. They specify an interface as a set of methods/attributes an implementer must have. Objects matching the interface conform implicitly (structural subtyping).

Now `mypy` or other type checkers can verify objects used with quacker.

In [6]:
from typing import Generic, Protocol, TypeVar

T = TypeVar("T")


class StackProtocol(Protocol[T]):
    def push(self, item: T) -> None: ...
    def pop(self) -> T: ...


class ArrayStack(Generic[T]):  # Generic base class
    def __init__(self) -> None:
        self._items: list[T] = []

    def push(self, item: T) -> None:
        self._items.append(item)

    def pop(self) -> T:
        return self._items.pop()


class ListStack(ArrayStack[T]):  # Inherit Generic[T]
    """Alternative implementation."""

    pass  # Same interface, different internal logic if needed


def process_stack(s: StackProtocol[int]) -> int:
    s.push(42)
    return s.pop()

## Inheritance

Inheritance is runtime polymorphism defined explicitly via class hierarchies. It is nominal typing (based on declared class relationships), less flexible than duck typing.

In [7]:
class Animal:
    def speak(self):
        return "Animal sound"


class Dog(Animal):
    def speak(self):
        return "Bark"


dog = Dog()
animal = Animal()


def make_animals_speak(animals: List[Animal]) -> List[str]:
    return [animal.speak() for animal in animals]


make_animals_speak([dog, animal])

['Bark', 'Animal sound']